In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

/Users/alex/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data Preparation

In [25]:
df = pd.read_csv('stocks_log_unscaled.csv')

# include stock dummy categorical variable for fix-effect regression
df['stock_dummy'] = df['ticker'].astype('category').cat.codes
df = df.rename(columns={'daily_volatility': 'daily_spread'})

df.head()

,Unnamed: 0,date,ticker,price,volume,return,high,low,open,avg_price,mkt_cap,pct_return,name_price_attention,daily_spread,log_return,log_attention,stock_dummy
0,0,2014-01-02,MSFT,37.16,30423757.0,-0.006683,37.40,37.10,37.350,37.25,308428000.0,-0.6683,3.15,0.30,-0.006705,1.147402,20
1,1,2014-01-03,MSFT,36.91,30944370.0,-0.006728,37.22,36.60,37.200,36.91,306353000.0,-0.6728,1.54,0.62,-0.006751,0.431782,20
2,2,2014-01-06,MSFT,36.13,43111355.0,-0.021133,36.89,36.11,36.850,36.50,299879000.0,-2.1133,2.24,0.78,-0.021359,0.806476,20
3,3,2014-01-07,MSFT,36.41,35478667.0,0.007750,36.49,36.21,36.325,36.35,302203000.0,0.7750,1.19,0.28,0.007720,0.173953,20
4,4,2014-01-08,MSFT,35.76,59452427.0,-0.017852,36.14,35.58,36.000,35.86,296808000.0,-1.7852,1.68,0.56,-0.018013,0.518794,20


In [26]:
df_scaled = pd.read_csv('stocks_log_norm_scaled.csv')
df_scaled = df_scaled.rename(columns={'daily_volatility': 'daily_spread'})
df_scaled.head()

,Unnamed: 0,date,permno,ticker,price,volume,return,high,low,open,avg_price,mkt_cap,pct_return,name_price_attention,volatility,industry,stock_dummy,log_return,log_name_price_attention
0,0,2014-01-02,10107,MSFT,-0.999221,1.117934,-0.006683,-0.992613,-1.004799,-0.997672,-0.998750,0.138220,-0.419196,-0.712226,-0.494860,Technology,20,-0.006705,1.423108
1,1,2014-01-03,10107,MSFT,-1.001527,1.148820,-0.006728,-0.994243,-1.009502,-0.999057,-1.001888,0.132557,-0.421781,-0.816796,-0.438358,Technology,20,-0.006751,0.932164
2,2,2014-01-06,10107,MSFT,-1.008721,1.870624,-0.021133,-0.997232,-1.014112,-1.002286,-1.005672,0.114890,-1.249333,-0.771331,-0.410107,Technology,20,-0.021359,1.175573
3,3,2014-01-07,10107,MSFT,-1.006139,1.417816,0.007750,-1.000854,-1.013171,-1.007130,-1.007056,0.121232,0.409965,-0.839528,-0.498392,Technology,20,0.007720,0.783902
4,4,2014-01-08,10107,MSFT,-1.012134,2.840056,-0.017852,-1.004023,-1.019098,-1.010129,-1.011579,0.106510,-1.060843,-0.807703,-0.448952,Technology,20,-0.018013,0.985817


In [24]:
anomalies_df = pd.read_csv('anomalies_dummies.csv')

# rename volatility as daily_volatility
anomalies_df = anomalies_df.rename(columns={'volatility': 'daily_spread'})
anomalies_df['stock_dummy'] = anomalies_df['ticker'].astype('category').cat.codes
anomalies_df.head()

,Unnamed: 0,date,permno,ticker,price,volume,return,high,low,open,...,industry,revenue,total_assets,earnings_per_share,book_to_market,dt_date,jan_dummy,monday_dummy,annual_rep_release,stock_dummy
0,0,2014-01-02,10107,MSFT,-0.999221,1.117934,-0.006683,-0.992613,-1.004799,-0.997672,...,Technology,NaN,NaN,NaN,NaN,2014-01-02,1,0,0,20
1,1,2014-01-03,10107,MSFT,-1.001527,1.148820,-0.006728,-0.994243,-1.009502,-0.999057,...,Technology,NaN,NaN,NaN,NaN,2014-01-03,1,0,0,20
2,2,2014-01-06,10107,MSFT,-1.008721,1.870624,-0.021133,-0.997232,-1.014112,-1.002286,...,Technology,NaN,NaN,NaN,NaN,2014-01-06,1,1,0,20
3,3,2014-01-07,10107,MSFT,-1.006139,1.417816,0.007750,-1.000854,-1.013171,-1.007130,...,Technology,NaN,NaN,NaN,NaN,2014-01-07,1,0,0,20
4,4,2014-01-08,10107,MSFT,-1.012134,2.840056,-0.017852,-1.004023,-1.019098,-1.010129,...,Technology,NaN,NaN,NaN,NaN,2014-01-08,1,0,0,20


In [8]:
df.describe()

/Users/alex/anaconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/alex/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Unnamed: 0,price,volume,return,high,low,open,avg_price,mkt_cap,pct_return,name_price_attention,daily_volatility,log_return,log_attention
count,70448.000000,70448.000000,7.044800e+04,70448.000000,70448.000000,70448.000000,70448.000000,70448.000000,7.044800e+04,70448.000000,70448.000000,70448.000000,70448.000000,7.044800e+04
mean,35223.500000,145.495397,1.157947e+07,0.000614,147.012658,143.910029,145.475873,145.461344,2.577778e+08,0.061384,14.115716,3.102629,0.000462,-inf
std,20336.730219,108.420623,1.685646e+07,0.017407,110.429143,106.300702,108.378896,108.347582,3.664491e+08,1.740688,15.396513,5.663514,0.017416,NaN
min,0.000000,21.349900,3.053580e+05,-0.238484,21.620000,21.270000,21.340000,21.445000,1.715150e+07,-23.848400,0.000000,0.090000,-0.272444,-inf
25%,17611.750000,80.590000,3.327002e+06,-0.006783,81.240000,80.000000,80.580000,80.613750,1.069959e+08,-0.678300,0.000000,1.040000,-0.006806,NaN
50%,35223.500000,125.940000,6.196272e+06,0.000619,127.090000,124.750000,125.900000,125.850000,1.683775e+08,0.061900,9.680000,1.870010,0.000619,2.270062e+00
75%,52835.250000,176.352508,1.294353e+07,0.008153,177.950000,174.610000,176.292492,176.281250,2.482573e+08,0.815300,21.840000,3.450000,0.008120,3.083743e+00
max,70447.000000,2238.750000,6.182376e+08,0.260449,2318.489990,2186.520020,2295.120120,2252.505005,3.081156e+09,26.044900,100.000000,323.879880,0.231468,4.605170e+00


In [9]:
df_scaled.describe()

,Unnamed: 0,permno,price,volume,return,high,low,open,avg_price,mkt_cap,pct_return,name_price_attention,volatility,stock_dummy,log_return,log_name_price_attention
count,70448.000000,70448.000000,7.044800e+04,7.044800e+04,70448.000000,7.044800e+04,7.044800e+04,7.044800e+04,70448.000000,7.044800e+04,7.044800e+04,7.044800e+04,7.044800e+04,70448.000000,70448.000000,70448.000000
mean,35223.500000,42556.107143,-2.969336e-16,1.936523e-17,0.000614,9.037110e-17,-1.291016e-16,-1.032813e-16,0.000000,-2.582031e-17,-2.138245e-17,-2.582031e-17,3.873047e-17,13.500000,0.000462,2.000092
std,20336.730219,28269.163826,1.000007e+00,1.000007e+00,0.017407,1.000007e+00,1.000007e+00,1.000007e+00,1.000007,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,8.077805,0.017416,1.379421
min,0.000000,10107.000000,-1.145044e+00,-6.688350e-01,-0.238484,-1.135511e+00,-1.153717e+00,-1.145396e+00,-1.144624,-6.566479e-01,-1.373592e+01,-9.168190e-01,-5.319401e-01,0.000000,-0.272444,0.000000
25%,17611.750000,17270.500000,-5.986486e-01,-4.895764e-01,-0.006783,-5.956138e-01,-6.012235e-01,-5.987912e-01,-0.598519,-4.114707e-01,-4.249406e-01,-9.168190e-01,-3.641986e-01,6.750000,-0.006806,0.000000
50%,35223.500000,34926.000000,-1.803673e-01,-3.193574e-01,0.000619,-1.804125e-01,-1.802450e-01,-1.806257e-01,-0.181005,-2.439655e-01,2.964425e-04,-2.881008e-01,-2.176436e-01,13.500000,0.000619,2.368373
75%,52835.250000,61063.000000,2.846076e-01,8.092260e-02,0.008153,2.801576e-01,2.888051e-01,2.843435e-01,0.284456,-2.598057e-02,4.331168e-01,5.016940e-01,6.133531e-02,20.250000,0.008120,3.128513
max,70447.000000,93436.000000,1.930693e+01,3.598990e+01,0.260449,1.966413e+01,1.921553e+01,1.983467e+01,19.447214,7.704748e+00,1.492726e+01,5.578204e+00,5.663966e+01,27.000000,0.231468,4.615121


### OLS Linear Regression 

Univariate Linear Regression is a simple linear regression model with a single explanatory variable. It is a straight line fit to the data. The model is defined as:

In [14]:
from statsmodels.formula.api import ols

model_1 = ols('Q("return") ~  name_price_attention + C(stock_dummy)', data=df).fit() 

model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Q("return")   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.814
Date:                Thu, 07 Mar 2024   Prob (F-statistic):            0.00529
Time:                        16:26:32   Log-Likelihood:             1.8544e+05
No. Observations:               70448   AIC:                        -3.708e+05
Df Residuals:                   70419   BIC:                        -3.706e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0006      0.000      1.620      0.105      -0.000       0.001
C(stock_dummy)[T.1]     -0.0004      0.000     -0.747      0.455      -0.001       0.001
C(stock_dummy)[T.2]     -0.0002      0.000     -0.497      0.619      -0.001       0.001
C(stock_dummy)[T.3]     -0.0002      0.000     -0.381      0.703      -0.001       0.001
C(stock_dummy)[T.4]     -0.0003      0.000     -0.594      0.553      -0.001       0.001
C(stock_dummy)[T.5]   4.916e-05      0.000      0.099      0.921      -0.001       0.001
C(stock_dummy)[T.6]     -0.0005      0.000     -0.949      0.343      -0.001       0.000
C(stock_dummy)[T.7]     -0.0006      0.000     -1.194      0.232      -0.002       0.000
C(stock_dummy)[T.8]     -0.0006      0.000     -1.174      0.240      -0.002       0.000
C(stock_dummy)[T.9]     -0.0002      0.000     -0.480      0.631      -0.001       0.001
C(stock_dummy)[T.10]    -0.0004      0.000     -0.727      0.467      -0.001       0.001
C(stock_dummy)[T.11]    -0.0004      0.000     -0.714      0.475      -0.001       0.001
C(stock_dummy)[T.12]    -0.0012      0.000     -2.374      0.018      -0.002      -0.000
C(stock_dummy)[T.13]    -0.0006      0.000     -1.274      0.203      -0.002       0.000
C(stock_dummy)[T.14]    -0.0004      0.000     -0.760      0.447      -0.001       0.001
C(stock_dummy)[T.15]    -0.0002      0.000     -0.339      0.735      -0.001       0.001
C(stock_dummy)[T.16]    -0.0004      0.000     -0.802      0.422      -0.001       0.001
C(stock_dummy)[T.17]    -0.0002      0.000     -0.482      0.630      -0.001       0.001
C(stock_dummy)[T.18]    -0.0007      0.000     -1.405      0.160      -0.002       0.000
C(stock_dummy)[T.19]    -0.0003      0.000     -0.625      0.532      -0.001       0.001
C(stock_dummy)[T.20]     0.0001      0.000      0.240      0.811      -0.001       0.001
C(stock_dummy)[T.21] -3.835e-05      0.001     -0.077      0.939      -0.001       0.001
C(stock_dummy)[T.22]    -0.0004      0.000     -0.871      0.384      -0.001       0.001
C(stock_dummy)[T.23]    -0.0002      0.000     -0.474      0.636      -0.001       0.001
C(stock_dummy)[T.24]     0.0008      0.000      1.636      0.102      -0.000       0.002
C(stock_dummy)[T.25]     0.0004      0.001      0.723      0.470      -0.001       0.001
C(stock_dummy)[T.26]    -0.0011      0.000     -2.149      0.032      -0.002   -9.27e-05
C(stock_dummy)[T.27]    -0.0006      0.000     -1.129      0.259      -0.002       0.000
name_price_attention   2.36e-05   4.94e-06      4.777      0.000    1.39e-05    3.33e-05
==============================================================================
Omnibus:                    15704.034   Durbin-Watson:                   2.103
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           767936.50

In the following model we include all control variables to see wether the performance of the model improves. 

In [27]:
model_2 = ols('Q("return") ~  name_price_attention + C(stock_dummy) + volume + daily_spread + mkt_cap + jan_dummy + monday_dummy + annual_rep_release', data=anomalies_df).fit() 

model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Q("return")   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.935
Date:                Thu, 07 Mar 2024   Prob (F-statistic):           2.23e-08
Time:                        16:48:19   Log-Likelihood:             1.8547e+05
No. Observations:               70448   AIC:                        -3.709e+05
Df Residuals:                   70413   BIC:                        -3.705e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0016      0.001      3.118      0.002       0.001       0.003
C(stock_dummy)[T.1]     -0.0012      0.001     -1.890      0.059      -0.002    4.55e-05
C(stock_dummy)[T.2]     -0.0010      0.001     -1.577      0.115      -0.002       0.000
C(stock_dummy)[T.3]     -0.0008      0.001     -1.208      0.227      -0.002       0.000
C(stock_dummy)[T.4]     -0.0011      0.001     -1.711      0.087      -0.002       0.000
C(stock_dummy)[T.5]     -0.0006      0.001     -1.017      0.309      -0.002       0.001
C(stock_dummy)[T.6]     -0.0007      0.001     -1.201      0.230      -0.002       0.000
C(stock_dummy)[T.7]     -0.0014      0.001     -2.214      0.027      -0.003      -0.000
C(stock_dummy)[T.8]     -0.0012      0.001     -1.989      0.047      -0.002   -1.78e-05
C(stock_dummy)[T.9]     -0.0010      0.001     -1.495      0.135      -0.002       0.000
C(stock_dummy)[T.10]    -0.0013      0.001     -2.048      0.041      -0.003   -5.62e-05
C(stock_dummy)[T.11]    -0.0012      0.001     -1.874      0.061      -0.002    5.55e-05
C(stock_dummy)[T.12]    -0.0022      0.001     -3.307      0.001      -0.004      -0.001
C(stock_dummy)[T.13]    -0.0007      0.001     -1.169      0.243      -0.002       0.000
C(stock_dummy)[T.14]    -0.0012      0.001     -2.018      0.044      -0.002   -3.48e-05
C(stock_dummy)[T.15]    -0.0008      0.001     -1.282      0.200      -0.002       0.000
C(stock_dummy)[T.16]    -0.0009      0.001     -1.488      0.137      -0.002       0.000
C(stock_dummy)[T.17]    -0.0011      0.001     -1.674      0.094      -0.002       0.000
C(stock_dummy)[T.18]    -0.0015      0.001     -2.360      0.018      -0.003      -0.000
C(stock_dummy)[T.19]    -0.0010      0.001     -1.575      0.115      -0.002       0.000
C(stock_dummy)[T.20]    -0.0005      0.001     -1.012      0.312      -0.002       0.000
C(stock_dummy)[T.21]    -0.0007      0.001     -1.059      0.290      -0.002       0.001
C(stock_dummy)[T.22]    -0.0012      0.001     -2.000      0.046      -0.002   -2.43e-05
C(stock_dummy)[T.23]    -0.0010      0.001     -1.585      0.113      -0.002       0.000
C(stock_dummy)[T.24]     0.0011      0.001      1.704      0.088      -0.000       0.002
C(stock_dummy)[T.25]    -0.0004      0.001     -0.612      0.541      -0.002       0.001
C(stock_dummy)[T.26]    -0.0016      0.001     -2.606      0.009      -0.003      -0.000
C(stock_dummy)[T.27]    -0.0014      0.001     -2.350      0.019      -0.003      -0.000
name_price_attention     0.0005   8.53e-05      5.991      0.000       0.000       0.001
volume                  -0.0006   9.62e-05     -6.404      0.000      -0.001      -0.000
daily_spread            -0.0002   8.26e-05     -1.883      0.060      -0.000    6.35e-06
mkt_cap                  0.0003      0.000      2.262    

The low R-squared values in combination with the high AIC and BIC values suggest that the models have very low to no predictive power.

**Test statistics**

Omnibus Test: The Omnibus test is a statistical test to check the skewness and kurtosis of the residuals. A significant result (Prob(Omnibus) close to 0, as in this case) indicates that the residuals are not normally distributed. In your results, the Omnibus statistic is very high, and the probability is 0.000, which suggests a departure from the normality assumption.


Skew: This measures the asymmetry of the residuals distribution. A skew close to 0, which is the case here (0.092), suggests that the residuals are fairly symmetrical.


Kurtosis: This measures the 'tailedness' of the residuals distribution. A kurtosis value significantly larger than 3 indicates a distribution with heavier tails than a normal distribution. Here, the kurtosis is 19.174, which is quite high, indicating heavy tails.


Durbin-Watson: This statistic tests for the presence of autocorrelation in the residuals. A value close to 2 suggests there is no first-order linear autocorrelation. With your Durbin-Watson value of 2.103, it appears that autocorrelation is not a concern.


Jarque-Bera (JB) Test: This is another test for normality in the residuals. Like the Omnibus test, if the probability of JB is low (as it is here, Prob(JB) = 0.00), it indicates that the residuals do not follow a normal distribution. The JB statistic is extremely high, reinforcing the result of the Omnibus test.
Condition Number: A high condition number (over 20-30) can indicate multicollinearity. In your results, the condition number is 600, which suggests potential multicollinearity issues in the model.

Given these test results it is clear that we need to transform our data to make it more suitable for linear regression. We start by performing a log transformation on the target variable.


OLS 